# The Maxis Task

### Task to do:
1.     Create data pipeline to ingest data from public website(airport DB: link), store data in raw form in Google Storage, and ingest the data into BigQuery for analysis purpose.
( link : https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat )
2.     Share snapshot of the codes, and provide screenshot of BigQuery showing how many airports there are in Malaysia.

3.     Material/Codes to be shared for reference. Publish in Git/Google Cloud Source Repo.

### packages to install before the program

In [ ]:
!pip install google-cloud-storage
!pip install google-cloud-bigquery
!pip install pyarrow

## The Airport Data Pipeline

### Download raw data file from URL

In [20]:
from google.cloud import storage
import pandas as pd 
import requests
import io

#Download file from url
url="https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat"
file = requests.get(url)
open('C:/Users/weionn.chong/Documents/Onn/Work/R&D/maxis_task/airport.dat', 'wb').write(myfile.content)
print("File download complete")

File download complete


### Upload raw data file to storage

In [22]:
#store raw data to google cloud storage
client = storage.Client.from_service_account_json('onn-research-dev-storage.json')   #initializing google storage with credentials
bucket = client.get_bucket('maxis_task')
blob = bucket.blob('airport.dat')
blob.upload_from_filename('airport.dat')
print("Upload to google storage complete")

Upload to google storage complete


### Data Transformation

In [63]:
#peek the data file from url
data = pd.read_csv('airport.dat', header = None)
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports


In [53]:
print(len(data))

7698


In [47]:
#Transformation
#Renaming the column names
data.rename(columns = {1: 'airport_name',
                       2: 'city',
                       3: 'country',
                       4: 'airport_code',
                       5: 'iso_code',
                       6: 'latitude',
                       7: 'longitude',
                       8: 'elevation',
                       9: 'timezone',
                       10: 'unknown',
                       11: 'continent/region',
                       12: 'type',
                       13: 'source'}, inplace=True)

In [59]:
#reset index and droping the initial index column
data = data.reset_index().drop(columns = 0)

### Upload dataframe to BigQuery table

In [62]:
from google.cloud import bigquery
#Initialize the project id and staging table destination
table_destination_firm = 'onn-research-dev.maxis_task.airport_db'

#initializing the client with google credentials
client = bigquery.Client.from_service_account_json('onn-research-dev-bigquery.json')

#Setting the schema for the bigquery table
job_config_airport = bigquery.LoadJobConfig(scheme=[
                        bigquery.SchemaField("no", "STRING"),
                        bigquery.SchemaField("airport_name", "STRING"),
                        bigquery.SchemaField("city", "STRING"),
                        bigquery.SchemaField("country", "STRING"),
                        bigquery.SchemaField("airport_code", "STRING"),
                        bigquery.SchemaField("iso_code", "STRING"),
                        bigquery.SchemaField("latitude", "STRING"),
                        bigquery.SchemaField("longitude", "STRING"),
                        bigquery.SchemaField("elevation", "STRING"),
                        bigquery.SchemaField("timezone", "STRING"),
                        bigquery.SchemaField("Unknown", "STRING"),
                        bigquery.SchemaField("continent/region", "STRING"),
                        bigquery.SchemaField("type", "STRING"),
                        bigquery.SchemaField("source", "STRING")],
                        write_disposition="WRITE_TRUNCATE"
                    )
#passing dataframe to bigquery table
job_airport = client.load_table_from_dataframe(data, table_destination_firm, job_config=job_config_airport)

job_airport.result()
print('Upload to bigquery table is completed')

## View Result (Extra)

### Show all airport in Malaysia and the total number

In [67]:
#run sql job to query all the airports in malaysia
sql = """
    SELECT distinct *
    FROM `onn-research-dev.maxis_task.airport_db`
    WHERE country = @country
"""
query_config = bigquery.QueryJobConfig(
                    query_parameters=[
                        bigquery.ScalarQueryParameter('country','STRING','Malaysia')
                    ])

result_my = client.query(sql, job_config=query_config).to_dataframe()

display(result_my) #diplay airport details in malaysia
print(f"Total number of airport in Malaysia is {len(result_my)}")  #number of airport in malaysia

,airport_name,city,country,airport_code,iso_code,latitude,longitude,elevation,timezone,unknown,continent_region,type,source
0,Bintulu Airport,Bintulu,Malaysia,BTU,WBGB,3.12385,113.019997,74,8,N,Asia/Kuala_Lumpur,airport,OurAirports
1,Kuching International Airport,Kuching,Malaysia,KCH,WBGG,1.48470,110.347000,89,8,N,Asia/Kuala_Lumpur,airport,OurAirports
2,Limbang Airport,Limbang,Malaysia,LMN,WBGJ,4.80830,115.010002,14,8,N,Asia/Kuala_Lumpur,airport,OurAirports
3,Marudi Airport,Marudi,Malaysia,MUR,WBGM,4.17898,114.329002,103,8,N,Asia/Kuala_Lumpur,airport,OurAirports
4,Miri Airport,Miri,Malaysia,MYY,WBGR,4.32201,113.987000,59,8,N,Asia/Kuala_Lumpur,airport,OurAirports
5,Sibu Airport,Sibu,Malaysia,SBW,WBGS,2.26160,111.985001,122,8,N,Asia/Kuala_Lumpur,airport,OurAirports
6,Lahad Datu Airport,Lahad Datu,Malaysia,LDU,WBKD,5.03225,118.323997,45,8,N,Asia/Kuala_Lumpur,airport,OurAirports
7,Kota Kinabalu International Airport,Kota Kinabalu,Malaysia,BKI,WBKK,5.93721,116.051003,10,8,N,Asia/Kuala_Lumpur,airport,OurAirports
8,Labuan Airport,Labuan,Malaysia,LBU,WBKL,5.30068,115.250000,101,8,N,Asia/Kuala_Lumpur,airport,OurAirports
9,Tawau Airport,Tawau,Malaysia,TWU,WBKW,4.32016,118.127998,57,8,N,Asia/Kuala_Lumpur,airport,OurAirports


Total number of airport in Malaysia is 40
